In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/home/evanvance/Code/School/network-anomaly-ai/reports/Monday-WorkingHours.pcap_report.csv")

In [30]:
benign_count =  df["Predictions"].value_counts().loc["BENIGN"]
malicious_count = df["Predictions"].value_counts().loc["MALICIOUS"]

In [31]:
print(f'''
Benign: {benign_count}
Malicious: {malicious_count}

Since we analyzed a capture that we know had no malicious interactions,
the models accuracy was {round(benign_count / df["Predictions"].count(), 4)}

''')


Benign: 328265
Malicious: 27467

Since we analyzed a capture that we know had no malicious interactions,
the models accuracy was 0.9228




In [ ]:
from sklearn.metrics import auc, precision_recall_curve, accuracy_score, precision_score, recall_score, roc_auc_score

def score_predictions(y_truth, y_predictions):
    score = {}

    precision, recall, _ = precision_recall_curve(y_truth, y_predictions)
    score["auprc"] = auc(recall, precision)
    # score["auc_roc"] = roc_auc_score(y_truth, y_predictions)
    score["precision"] = precision_score(y_truth, y_predictions)
    score["recall"] = recall_score(y_truth, y_predictions)
    score["accuracy"] = accuracy_score(y_truth, y_predictions)

    return score

In [33]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [34]:
df["Labels"] = 0

df["Predictions"] = le.fit_transform(df["Predictions"])

In [ ]:
score_predictions(df["Predictions"], df["Labels"])

/home/evanvance/miniconda3/envs/ids-ai/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/evanvance/miniconda3/envs/ids-ai/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'auprc': 0.5, 'precision': 0.0, 'recall': 0.0, 'accuracy': 0.9227873792630407}